In [1]:
import os
import time
import openpyxl
from openai import OpenAI
from pathlib import Path

# OpenAI API 客户端（阿里云千问）
client = OpenAI(
    api_key="sk-038c3b8e36ce43e4a94156257a766bc4",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

print(os.getcwd())


c:\Users\Nack\Desktop\Nack-graduation-project\LLM\Qwen


In [2]:
IMAGE_FOLDER = "images"  # 请修改为你的图片文件夹路径
OUTPUT_EXCEL = "images/image_descriptions.xlsx"  # 结果保存的Excel文件

# 支持的图片格式
VALID_EXTENSIONS = {".jpg", ".png", ".jpeg"}

# 创建 Excel 工作簿和工作表
wb = openpyxl.Workbook()
ws = wb.active
ws.append(["图片路径", "图片描述"])  # 添加表头


In [ ]:
import base64

def describe_image(image_path, client):
    """调用 API 获取图片描述"""
    try:
        # 读取图片并进行 Base64 编码
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

        # 发送请求
        response = client.chat.completions.create(
            model="qwen2.5-vl-72b-instruct",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": "你是一个机械制造领域的大师，现在有一个待安装SMA的盒子，共有20空位。空位中已有2个安装完成SMA，请你为剩下的空位规划安装顺序，使得安装效率最高。告诉我安装顺序坐标。"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}
            ]
        )

        # 获取返回结果
        print(f"response: {response}")
        return response.choices[0].message.content
    except Exception as e:
        print(f"处理 {image_path} 时出错: {e}")
        return "API 调用失败"


In [4]:
# 遍历文件夹中的所有图片
for image_file in os.listdir(IMAGE_FOLDER):
    image_path = os.path.join(IMAGE_FOLDER, image_file)
    image_path = os.path.abspath(image_path)  

    # 检查是否是图片文件
    if Path(image_path).suffix.lower() in VALID_EXTENSIONS:
        print(f"处理图片: {image_path}")
        description = describe_image(image_path, client)
        
        # 存入 Excel
        ws.append([image_path, description])
        
        # 适当延迟，避免 API 速率限制
        time.sleep(1)

# 保存 Excel 文件
wb.save(OUTPUT_EXCEL)
print(f"所有图片处理完成，结果已保存至 {OUTPUT_EXCEL}")

处理图片: c:\Users\Nack\Desktop\Nack-graduation-project\LLM\Qwen\images\图1.jpg
response: ChatCompletion(id='chatcmpl-56dbba88-3693-9d91-a617-ad20ec389e43', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='根据图片中的信息，我们可以进行以下分析：\n\n1. **空位总数**：图片中有20个空位。\n2. **已安装数量**：已经有2个空位安装了SMA（左上角和中间偏右的位置）。\n3. **待安装数量**：还有18个空位需要安装SMA。\n\n为了确保安装效率最高，可以按照从左到右、从上到下的顺序进行安装。这样可以减少机械臂的移动距离，提高安装速度。\n\n### 安装顺序坐标：\n假设每个空位用一个坐标表示，从左上角开始编号为(1,1)，依次向右和向下编号。\n\n- 已安装位置：(1,1) 和 (4,3)\n- 待安装位置按顺序排列如下：\n\n```\n(1,2), (1,3), (1,4), (1,5)\n(2,1), (2,2), (2,3), (2,4), (2,5)\n(3,1), (3,2), (3,3), (3,4), (3,5)\n(4,1), (4,2), (4,4), (4,5)\n(5,1), (5,2), (5,3), (5,4), (5,5)\n```\n\n这样安排可以确保机械臂在安装过程中尽量减少不必要的移动，从而提高安装效率。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1743762320, model='qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completio